# Getting Started

The purpose of the present `Getting Started` section is to give a quick overview of the main objects, methods and functions of the Python `iode` library.
To get a more detailed presentation of all capabilities of `iode`, read the next sections of the tutorial.
    
The [API Reference](../api.rst#api-reference) section of the documentation give you the list of all objects, methods and functions with their individual documentation and examples.

The [Equivalence IODE Report Commands and IODE Python](../equivalence.rst#equivalence-iode-report-commands-and-iode-python) section contains *equivalence tables* between the IODE report syntax and the Python `iode` syntax.

To use the Python `iode` library, the first thing to do is to import objects and functions you need from it:

In [1]:
import numpy as np
from iode import (SAMPLE_DATA_DIR, comments, equations, identities, lists, scalars, 
                  tables, variables, EqMethod, Sample)

To know the version of the `iode` library installed on your machine, type:

In [2]:
from iode import __version__
__version__

'7.0.0'

To get the list of objects and functions available in the `iode` li library, use the Python function `dir()`:

In [3]:
import iode
dir(iode)

['AdjustmentMethod',
 'Comments',
 'EQUATION_METHODS_LIST',
 'EQ_TEST_NAMES',
 'ESTIMATION_EPS',
 'ESTIMATION_MAXIT',
 'EXPORT_FORMATS',
 'EqMethod',
 'EqTest',
 'Equation',
 'Equations',
 'ExportFormats',
 'FileType',
 'GRAPHS_COLORS',
 'HighToLowType',
 'IMPORT_FORMATS',
 'IODE_DATABASE_TYPE_NAMES',
 'IODE_FILE_TYPES',
 'IODE_FILE_TYPE_NAMES',
 'IODE_FONTS',
 'IODE_LANGUAGES_LIST',
 'Identities',
 'Identity',
 'ImportFormats',
 'IodeFileType',
 'IodeType',
 'Lists',
 'LowToHighMethod',
 'LowToHighType',
 'NA',
 'PERIODICITY_LIST',
 'PRINT_FORMATS',
 'Path',
 'Period',
 'SAMPLE_DATA_DIR',
 'SIMULATION_INITIALIZATION_METHODS',
 'SIMULATION_SORT_ALGORITHMS',
 'Sample',
 'Scalar',
 'Scalars',
 'Simulation',
 'SimulationInitialization',
 'SimulationSort',
 'Table',
 'TableCellAlign',
 'TableCellFont',
 'TableCellType',
 'TableGraphAlign',
 'TableGraphAxis',
 'TableGraphGrid',
 'TableGraphType',
 'TableLang',
 'TableLineType',
 'Tables',
 'Variables',
 'VarsMode',
 'WriteFileExt',
 '__all_

To print the documentation of an object, method or function in a Python interactive console, use the `help()` function:

In [4]:
# ---- print documentation of a function or method ----
help(equations.load)

Help on method load in module iode.iode_cython:

load(filepath: 'str') method of iode.iode_cython.Equations instance
    load(self, filepath: str)
    
    Load objects stored in file 'filepath' into the current database.
    Erase the database before to load the file.
    
    Parameters
    ----------
    filepath: str
        path to the file to load
    
    Examples
    --------
    >>> from iode import SAMPLE_DATA_DIR
    >>> from iode import comments, variables
    >>> comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")         # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    Loading .../fun.cmt
    317 objects loaded 
    >>> len(comments)
    317
    
    >>> variables.load(f"{SAMPLE_DATA_DIR}/fun.var")        # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    Loading .../fun.var
    394 objects loaded
    >>> len(variables)
    394



## Load IODE objects

To load IODE objects from a binary file (i.e. with extension `.cmt`, `.eqs`, `.idt`, `.lst`, `.scl`, `.tbl`, `.var`) or from an ASCII file (i.e. with extension `.ac`, `.ae`, `.ai`, `.al`, `.as`, `.at`, `.av`), use the [load()](../_generated/iode.Comments.load.rst#iode.Comments.load) method of the corresponding object. For example:

In [5]:
# ---- load equations, identities, scalars and variables ----
# Note: test binary and ASCII 'fun' files are located in the 'SAMPLE_DATA_DIR' 
#       directory of the 'iode' package
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

# ---- print the number of objects present in the above workspaces ----
len(comments), len(equations), len(identities), len(lists), len(scalars), len(tables), len(variables)

Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.cmt
317 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.eqs
274 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.idt
48 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.lst
17 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.scl
161 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.tbl
46 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.var
394 objects loaded


(317, 274, 48, 17, 161, 46, 394)

## Save IODE objects

To save the content of a workspace (or a subset of a workspace), use the [save()](../_generated/iode.Variables.save.rst#iode.Variables.save) method:

In [6]:
# ---- save workspace (or subset) ----
# save the whole workspace
equations.save('equations.eqs')

# save only a subset of the global variables workspace
vars_subset = variables[["ACAF", "ACAG", "AQC", "BQY", "BVY"]]
vars_subset.save('variables_subset.av')    

print("Check content of the variables_subset.av file:\n")
with open("variables_subset.av", "r") as f:
    print(f.read())
print()

Saving c:\usr\Projects\iode-1\doc\source\tutorial\equations.eqs
274 objects saved
Saving c:\usr\Projects\iode-1\doc\source\tutorial\variables_subset.av
Check content of the variables_subset.av file:

sample 1960Y1 2015Y1
ACAF na na na na na na na na na na 1.2130001 5.2020001 9.184 8.0790005 11.332 13.518001 15.784 16.544001 21.489 20.281 21.277 32.417999 24.446999 27.025002 24.504 27.560999 25.542 27.499001 25.353001 17.165001 23.771 26.240999 30.159 34.661999 8.1610022 -13.130997 32.171001 39.935799 29.645657 13.530404919696 10.0466107922005 2.86792273645546 -0.929212509051645 -6.09156498756888 -14.5820944628981 -26.5387895697886 -28.9872879825975 -33.3784257842954 -38.4095177823974 -37.4635096412738 -37.8274288322944 -44.5447926335432 -55.5592898172187 -68.8946543226201 -83.3406251108009 -96.4104198284833 
ACAG na na na na na na na na na na -11.028999 -15.847 -19.288002 -21.814999 -25.447002 -24.618999 -27.770998 -28.839001 -29.434998 -30.411001 -30.353001 -41.060997 -31.178001 -32.6

## Working with workspaces

To get the list of objects names present in a workspace, use the [names](../_generated/iode.Comments.names.rst#iode.Comments.names) attribute of the workspace. 
For example:

In [7]:
# get the list of all IODE lists
lists.names

['COPY',
 'COPY0',
 'COPY1',
 'ENDO',
 'ENDO0',
 'ENDO1',
 'ENVI',
 'IDT',
 'MAINEQ',
 'MYLIST',
 'TOTAL',
 'TOTAL0',
 'TOTAL1',
 'XENVI',
 'XSCENARIO',
 '_SCAL',
 '_SEARCH']

To check if a name is present in a workspace, use the `in` operator. 
For example:

In [8]:
if 'ENVI' in lists:
    print("The 'ENVI' IODE list exists")
else:
    print("'ENVI' IODE list not found")

The 'ENVI' IODE list exists


To iterate over names of a workspace, simply use the Python syntax for the *for loop*:

In [9]:
print("Iterate over all IODE lists names in the Lists workspace:")
for name in lists:
    print(name)

Iterate over all IODE lists names in the Lists workspace:
COPY
COPY0
COPY1
ENDO
ENDO0
ENDO1
ENVI
IDT
MAINEQ
MYLIST
TOTAL
TOTAL0
TOTAL1
XENVI
XSCENARIO
_SCAL
_SEARCH


To get the current used sample for the Variables, use the [sample](../_generated/iode.Variables.sample.rst#iode.Variables.sample) attribute of the [variables](../_generated/iode.Variables.rst#iode.Variables) workspace:

In [10]:
# current used sample
variables.sample

Sample("1960Y1:2015Y1")

### Get - add - update - delete IODE object(s)

In a similar way to Python dictionaries, you can [get](../_generated/iode.Equations.__getitem__.rst), [add](../_generated/iode.Equations.__setitem__.rst), [update](../_generated/iode.Equations.__setitem__.rst) and [delete](../_generated/iode.Equations.__delitem__.rst) IODE objects in a workspace using the `[]` operator.

* To extract an IODE object from a workspace, use the syntax: `my_obj = workspace[name]`.
* To add an IODE object to a workspace, use the syntax: `workspace[new_name] = new_obj`.
* To update an IODE object in a workspace, use the syntax: `workspace[name] = new_value`.
* To delete an IODE object from a workspace, use the syntax: `del workspace[name]`.

#### Comments

Add one comment:

In [11]:
comments["NEW"] = "A new comment"
comments["NEW"]

'A new comment'

Update a comment:

In [12]:
comments["NEW"] = "New Value"
comments["NEW"]

'New Value'

Delete a comment:

In [13]:
comments.get_names("A*")

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [14]:
del comments["ACAF"]
comments.get_names("A*")

['ACAG', 'AOUC', 'AQC']

#### Equations

Add one equation:

In [15]:
equations["TEST"] = "TEST := 0"
equations["TEST"]

Equation(endogenous = 'TEST',
         lec = 'TEST := 0',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

Update an equation:

In [16]:
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+\nacaf4*(TIME=1995)',
         method = 'LSQ',
         from_period = '1980Y1',
         to_period = '1996Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

In [17]:
# update only the LEC
equations["ACAF"] = "(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)"
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)',
         method = 'LSQ',
         from_period = '1980Y1',
         to_period = '1996Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

In [18]:
# update block and sample of a block of equations to estimation (dictionary)
estim_sample = "2000Y1:2010Y1"
block = "ACAF; ACAG; AOUC"
for eq_name in block.split(';'):
    equations[eq_name] = {"sample": estim_sample, "block": block}

(equations["ACAF"].sample, equations["ACAG"].sample, equations["AOUC"].sample)

(Sample("2000Y1:2010Y1"), Sample("2000Y1:2010Y1"), Sample("2000Y1:2010Y1"))

In [19]:
(equations["ACAF"].block, equations["ACAG"].block, equations["AOUC"].block)

('ACAF; ACAG; AOUC', 'ACAF; ACAG; AOUC', 'ACAF; ACAG; AOUC')

In [20]:
# update lec, method, sample and block
equations["ACAF"].lec = "(ACAF/VAF[-1]) := acaf2 * GOSF[-1] + acaf4 * (TIME=1995)"
equations["ACAF"].method = EqMethod.MAX_LIKELIHOOD
# new equation sample is from 1990Y1 to the last year of Variables
equations["ACAF"].sample = "1990Y1:"
equations["ACAF"].block = "ACAF"
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) := acaf2 * GOSF[-1] + acaf4 * (TIME=1995)',
         method = 'MAX_LIKELIHOOD',
         from_period = '1990Y1',
         to_period = '2015Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

Delete an equation:

In [21]:
equations.get_names("A*")

['ACAF', 'ACAG', 'AOUC']

In [22]:
del equations["ACAF"]
equations.get_names("A*")

['ACAG', 'AOUC']

#### Identities

Add one identity:

In [23]:
identities["BDY"] = "YN - YK"
identities["BDY"]

Identity('YN - YK')

Update an identity:

In [24]:
identities["AOUC"]

Identity('((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+VAFF))[-1]+PM*(VM/(VM+VAFF))[-1]')

In [25]:
identities["AOUC"] = '(WCRH / WCRH[1990Y1]) * (VAFF / (VM+VAFF))[-1] + PM * (VM / (VM+VAFF))[-1]'
identities["AOUC"]

Identity('(WCRH / WCRH[1990Y1]) * (VAFF / (VM+VAFF))[-1] + PM * (VM / (VM+VAFF))[-1]')

Delete an identity:

In [26]:
identities.get_names("W*")

['W', 'WBGR', 'WCRH', 'WMINR', 'WO']

In [27]:
del identities["W"]
identities.get_names("W*")

['WBGR', 'WCRH', 'WMINR', 'WO']

#### Lists

Add one list:

In [28]:
# --- by passing a string
lists["A_VAR"] = "ACAF;ACAG;AOUC;AOUC_;AQC"
lists["A_VAR"]

['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [29]:
# --- by passing a Python list
b_vars = variables.get_names("B*")
b_vars

['BENEF', 'BQY', 'BRUGP', 'BVY']

In [30]:
lists["B_VAR"] = b_vars
lists["B_VAR"]

['BENEF', 'BQY', 'BRUGP', 'BVY']

Update a list:

In [31]:
# --- by passing a string
lists["A_VAR"] = "ACAF;ACAG;AOUC;AQC"
lists["A_VAR"]

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [32]:
# --- by passing a Python list
b_y_vars = variables.get_names("B*Y")
b_y_vars

['BQY', 'BVY']

In [33]:
lists["B_VAR"] = b_y_vars
lists["B_VAR"]

['BQY', 'BVY']

Delete a list:

In [34]:
lists.get_names("C*")

['COPY', 'COPY0', 'COPY1']

In [35]:
del lists["COPY"]
lists.get_names("C*")

['COPY0', 'COPY1']

#### Scalars

Add one scalar:

In [36]:
# 1. default relax to 1.0
scalars["a0"] = 0.1
scalars["a0"]

Scalar(0.1, 1, na)

In [37]:
# 2. value + relax
scalars["a1"] = 0.1, 0.9
scalars["a1"]

Scalar(0.1, 0.9, na)

Update a scalar:

In [38]:
scalars["acaf1"]

Scalar(0.0157684, 1, 0.00136871)

In [39]:
# only update the value
scalars["acaf1"] = 0.8
scalars["acaf1"]

Scalar(0.8, 1, 0.00136871)

In [40]:
# update value and relax (tuple)
scalars["acaf2"] = 0.8, 0.9
scalars["acaf2"]

Scalar(0.8, 0.9, na)

In [41]:
# update value and relax (list)
scalars["acaf2"] = (0.7, 0.8)
scalars["acaf2"]

Scalar(0.7, 0.8, na)

In [42]:
# update value and relax (dictionary)
scalars["acaf3"] = {"relax": 0.9, "value": 0.8}
scalars["acaf3"]

Scalar(0.8, 0.9, 0.87301)

In [43]:
# update value and/or relax (Scalar object)
# NOTE: the standard deviation (std) cannot be changed manually
scalars["acaf4"]

Scalar(-0.00850518, 1, 0.0020833)

In [44]:
scalars["acaf4"].value = 0.8
scalars["acaf4"].relax = 0.9
scalars["acaf4"]

Scalar(0.8, 0.9, 0.0020833)

Delete a scalar:

In [45]:
scalars.get_names("a*")

['a0', 'a1', 'acaf1', 'acaf2', 'acaf3', 'acaf4']

In [46]:
del scalars["acaf4"]
scalars.get_names("a*")

['a0', 'a1', 'acaf1', 'acaf2', 'acaf3']

#### Tables

Create an add a new table:

In [47]:
# 1. specify list of line titles and list of LEC expressions
lines_titles = ["GOSG:", "YDTG:", "DTH:", "DTF:", "IT:", "YSSG+COTRES:", "RIDG:", "OCUG:"]
lines_lecs = ["GOSG", "YDTG", "DTH", "DTF", "IT", "YSSG+COTRES", "RIDG", "OCUG"]
tables["TABLE_CELL_LECS"] = {"nb_columns": 2, "table_title": "New Table", "lecs_or_vars": lines_lecs,
                             "lines_titles": lines_titles, "mode": True, "files": True, "date": True}
tables["TABLE_CELL_LECS"]

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 16
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [48]:
# 2. specify list of variables
vars_list = ["GOSG", "YDTG", "DTH", "DTF", "IT", "YSSG", "COTRES", "RIDG", "OCUG", "$ENVI"]
tables["TABLE_VARS"] = {"nb_columns": 2, "table_title": "New Table", "lecs_or_vars": vars_list,
                        "mode": True, "files": True, "date": True}
tables["TABLE_VARS"]

DIVIS | 1                                                                    |        
TITLE |                                  "New Table"                                  
----- | ------------------------------------------------------------------------------
CELL  | ""                                                                   |   "#S" 
----- | ------------------------------------------------------------------------------
CELL  | "Bruto exploitatie-overschot: overheid (= afschrijvingen)."          |    GOSG
CELL  | "Overheid: geïnde indirecte belastingen."                            |    YDTG
CELL  | "Totale overheid: directe belasting van de gezinnen."                |     DTH
CELL  | "Totale overheid: directe vennootschapsbelasting."                   |     DTF
CELL  | "Totale indirecte belastingen."                                      |      IT
CELL  | "Globale overheid: ontvangen sociale zekerheidsbijdragen."           |    YSSG
CELL  | "Cotisation de responsabilisation."

Update a table:

In [49]:
tables["TABLE_CELL_LECS"]               

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 16
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [50]:
tables["TABLE_CELL_LECS"].graph_axis

'VALUES'

In [51]:
from iode import TableGraphAxis
# set graph axis type
tables["TABLE_CELL_LECS"].graph_axis = TableGraphAxis.SEMILOG
tables["TABLE_CELL_LECS"].graph_axis

'SEMILOG'

In [52]:
# get the first line
tables["TABLE_CELL_LECS"][0]

New Table

In [53]:
# get the last line
tables["TABLE_CELL_LECS"][-1]

<DATE>

In [54]:
# delete last line
del tables["TABLE_CELL_LECS"][-1]
tables["TABLE_CELL_LECS"]

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 

nb lines: 15
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

In [55]:
# get index of line containing YSSG+COTRES
index = tables["TABLE_CELL_LECS"].index("YSSG+COTRES")
index

9

In [56]:
tables["TABLE_CELL_LECS"][index]

('"YSSG+COTRES:"', 'YSSG+COTRES')

In [57]:
# get line type
tables["TABLE_CELL_LECS"][index].line_type

'CELL'

In [58]:
# get line graph type
tables["TABLE_CELL_LECS"][index].graph_type

'LINE'

In [59]:
# know if axis is left
tables["TABLE_CELL_LECS"][index].axis_left

True

In [60]:
# update cells
# double quotes "    -> STRING cell
# no double quotes   -> LEC cell
tables["TABLE_CELL_LECS"][index] = ('"YSSG:"', 'YSSG')
tables["TABLE_CELL_LECS"][index]

('"YSSG:"', 'YSSG')

In [61]:
# insert a new title line surrounded by two separator lines
tables["TABLE_CELL_LECS"].insert(index + 1, '-')
tables["TABLE_CELL_LECS"].insert(index + 2, "New Title")
tables["TABLE_CELL_LECS"].insert(index + 3, '-')
tables["TABLE_CELL_LECS"]

DIVIS | 1       |     
TITLE |  "New Table"  
----- | --------------
CELL  | ""      | "#S"
----- | --------------
CELL  | "GOSG:" | GOSG
CELL  | "YDTG:" | YDTG
CELL  | "DTH:"  |  DTH
CELL  | "DTF:"  |  DTF
CELL  | "IT:"   |   IT
CELL  | "YSSG:" | YSSG
----- | --------------
TITLE |  "New Title"  
----- | --------------
CELL  | "RIDG:" | RIDG
CELL  | "OCUG:" | OCUG
----- | --------------
MODE  | 
FILES | 

nb lines: 18
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

In [62]:
# append a new sepatator line
tables["TABLE_CELL_LECS"] += '-'
tables["TABLE_CELL_LECS"]

DIVIS | 1       |     
TITLE |  "New Table"  
----- | --------------
CELL  | ""      | "#S"
----- | --------------
CELL  | "GOSG:" | GOSG
CELL  | "YDTG:" | YDTG
CELL  | "DTH:"  |  DTH
CELL  | "DTF:"  |  DTF
CELL  | "IT:"   |   IT
CELL  | "YSSG:" | YSSG
----- | --------------
TITLE |  "New Title"  
----- | --------------
CELL  | "RIDG:" | RIDG
CELL  | "OCUG:" | OCUG
----- | --------------
MODE  | 
FILES | 
----- | --------------

nb lines: 19
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

Delete a table:

In [63]:
tables.get_names("G*")

['GAP', 'GDP', 'GFR', 'GFRLEVEL', 'GFRPC', 'GROWTH']

In [64]:
del tables["GFRLEVEL"]
tables.get_names("G*")

['GAP', 'GDP', 'GFR', 'GFRPC', 'GROWTH']

#### Variables

Get variable:

In [65]:
# get the value for a single period -> float 
variables["ACAF", "1990Y1"]

23.771

In [66]:
# get the values for all periods -> Variables
variables["ACAF"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	    na	    na	    na	    na	    na	    na	...	-37.46	-37.83	-44.54	-55.56	-68.89	-83.34	-96.41

In [67]:
# get the values for a range of periods -> Variables
variables["ACAF", "1990Y1:2000Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1990Y1:2000Y1
mode: LEVEL

name	1990Y1	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1	1996Y1	1997Y1	1998Y1	1999Y1	2000Y1
ACAF	 23.77	 26.24	 30.16	 34.66	  8.16	-13.13	 32.17	 39.94	 29.65	 13.53	 10.05

In [68]:
# get the values for a list non-contiguous periods -> pandas Series 
variables["ACAF", ["1990Y1", "1995Y1", "2000Y1"]]

time
1990Y1    23.771000
1995Y1   -13.130997
2000Y1    10.046611
Name: ACAF, dtype: float64

Add one variable:

In [69]:
# 1) same value for all periods
variables["A0"] = np.nan
variables["A0"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A0  	    na	    na	    na	    na	    na	    na	...	    na	    na	    na	    na	    na	    na	    na

In [70]:
# 2) vector (list) containing a specific value for each period
variables["A1"] = list(range(variables.nb_periods))
variables["A1"]                     

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A1  	  0.00	  1.00	  2.00	  3.00	  4.00	  5.00	...	 49.00	 50.00	 51.00	 52.00	 53.00	 54.00	 55.00

In [71]:
# 3) LEC expression
variables["A2"] = "t + 10"
variables["A2"]                     

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A2  	 10.00	 11.00	 12.00	 13.00	 14.00	 15.00	...	 59.00	 60.00	 61.00	 62.00	 63.00	 64.00	 65.00

In [72]:
# 4) another variable
variables["A3"] = variables["ACAF"]
variables["A3"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A3  	    na	    na	    na	    na	    na	    na	...	-37.46	-37.83	-44.54	-55.56	-68.89	-83.34	-96.41

Update a variable.

Set one value of a variable for a specific period:

In [73]:
variables["ACAG", "1990Y1"]

-28.1721855713507

In [74]:
variables["ACAG", "1990Y1"] = -28.2
variables["ACAG", "1990Y1"]

-28.2

Update all values of a variable:

In [75]:
variables["ACAF"]                   

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	    na	    na	    na	    na	    na	    na	...	-37.46	-37.83	-44.54	-55.56	-68.89	-83.34	-96.41

In [76]:
# 1) same value for all periods
variables["ACAF"] = np.nan
variables["ACAF"]                   

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	    na	    na	    na	    na	    na	    na	...	    na	    na	    na	    na	    na	    na	    na

In [77]:
# 2) vector (list) containing a specific value for each period
variables["ACAF"] = list(range(variables.nb_periods))
variables["ACAF"]                   

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	  0.00	  1.00	  2.00	  3.00	  4.00	  5.00	...	 49.00	 50.00	 51.00	 52.00	 53.00	 54.00	 55.00

In [78]:
# 3) LEC expression
variables["ACAF"] = "t + 10"
variables["ACAF"]                   

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	 10.00	 11.00	 12.00	 13.00	 14.00	 15.00	...	 59.00	 60.00	 61.00	 62.00	 63.00	 64.00	 65.00

In [79]:
# 4) another variable
variables["ACAG"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAG	    na	    na	    na	    na	    na	    na	...	 27.23	 28.25	 29.28	 30.32	 31.37	 32.42	 33.47

In [80]:
variables["ACAF"] = variables["ACAG"]
variables["ACAF"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF	    na	    na	    na	    na	    na	    na	...	 27.23	 28.25	 29.28	 30.32	 31.37	 32.42	 33.47

Set the values for range of periods:

In [81]:
# 1) variable(periods) = same value for all periods
variables["ACAF", "1991Y1:1995Y1"] = 0.0
variables["ACAF", "1991Y1:1995Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF	  0.00	  0.00	  0.00	  0.00	  0.00

In [82]:
# 2) variable(periods) = vector (list) containing a specific value for each period
variables["ACAF", "1991Y1:1995Y1"] = [1., 2., 3., 4., 5.]
variables["ACAF", "1991Y1:1995Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF	  1.00	  2.00	  3.00	  4.00	  5.00

In [83]:
# 3) variable(periods) = LEC expression
variables["ACAF", "1991Y1:1995Y1"] = "t + 10"
variables["ACAF", "1991Y1:1995Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF	 41.00	 42.00	 43.00	 44.00	 45.00

In [84]:
# 4) variable(periods) = Variables object
variables["ACAG", "1991Y1:1995Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAG	-30.93	-40.29	-43.16	-16.03	-41.85

In [85]:
variables["ACAF", "1991Y1:1995Y1"] = variables["ACAG", "1991Y1:1995Y1"]
variables["ACAF", "1991Y1:1995Y1"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF	-30.93	-40.29	-43.16	-16.03	-41.85

Set the values for a list of non-contiguous periods:

In [86]:
values = [1.0, 3.0, 5.0]
variables["ACAF", ["1991Y1", "1993Y1", "1995Y1"]] = values
variables["ACAF", ["1991Y1", "1993Y1", "1995Y1"]]

time
1991Y1    1.0
1993Y1    3.0
1995Y1    5.0
Name: ACAF, dtype: float64

Delete a variable:

In [87]:
variables.get_names("A*")

['A0', 'A1', 'A2', 'A3', 'ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [88]:
del variables["ACAF"]
variables.get_names("A*")

['A0', 'A1', 'A2', 'A3', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

### Workspace subsets

IODE workspaces can contains a lot objects and it can be sometimes easier to work on a subset of the objects present in a workspace. To get a subset of an IODE workspace, a *pattern* can be passed to the `[]` operator. 
A (sub-)`pattern` is a list of characters representing a group of object names. It includes some special characters which have a special meaning:

* `*` : any character sequence, even empty
* `?` : any character (one and only one)
* `@` : any alphanumerical char [A-Za-z0-9]
* `&` : any non alphanumerical char
* `|` : any alphanumeric character or none at the beginning and end of a string 
* `!` : any non-alphanumeric character or none at the beginning and end of a string 
* `\` : escape the next character

The *pattern* can contain sub-patterns, as well as, object names. The sub-patterns and object names are separated by a *separator* character which is either:

* a whitespace `' '`
* a comma `,`
* a semi-colon `;`
* a tabulation `\t`
* a newline `\n`

Note that the *pattern* can contain references to IODE lists which are prefixed with the symbol `$`.

<div class="alert alert-info">

**Note**

When an IODE object is added, updated or deleted from a subset of a workspace, the change is also applied to the global workspace. For example, if an equation is added to the subset of the *equations* workspace, the the change is also applied to the global *equations* database.

To create an isolate subset of a workspace, use the [copy](../_generated/iode.Equations.copy.rst#iode.Equations.copy) method. This method returns a new workspace in which each object is a copy of the original object from the global workspace. Any change made to the *copied subset* will not be applied to the global workspace. This can be useful for example if you want to save previous values of scalars before estimating an equation or a block of equations and then restore the original values if the estimated values are not satisfying.

</div>


#### Comments - Equations - Identities - Lists - Scalars - Tables

Let's first begin by reloading the example workspace for all IODE types:

In [89]:
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.cmt
317 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.eqs
274 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.idt
48 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.lst
17 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.scl
161 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.tbl
46 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.var
394 objects loaded


In [90]:
# 1) get subset using a pattern
comments_subset = comments["A*"]
comments_subset.names

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [91]:
# 2) add a comment to the subset
comments_subset["A0"] = "New Comment"
comments_subset["A0"]

'New Comment'

In [92]:
# --> new comment also appears in the global workspace
"A0" in comments, comments["A0"]

(True, 'New Comment')

In [93]:
# 3) update a comment in the subset
comments_subset["A0"] = "Updated Comment"
comments_subset["A0"]

'Updated Comment'

In [94]:
# --> comment is also updated in the global workspace
comments["A0"]

'Updated Comment'

In [95]:
# delete comment from the subset
del comments_subset["A0"]
comments_subset.names

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [96]:
# NOTE: the comment has also been deleted from the global database
"A0" in comments

False

In [97]:
comments.get_names("A*")

['ACAF', 'ACAG', 'AOUC', 'AQC']

#### Variables

In [98]:
# 1) get subset using a pattern
vars_subset = variables["A*"]
vars_subset.names

['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [99]:
# get the values for a specific period -> Variables
variables["A*", "1990Y1"]

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1990Y1:1990Y1
mode: LEVEL

 name	1990Y1
ACAF 	 23.77
ACAG 	-28.17
AOUC 	  1.00
AOUC_	  0.94
AQC  	  1.00

In [100]:
# get the variable values for range of periods -> Variables
variables["A*", "1990Y1:2000Y1"]

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1990Y1:2000Y1
mode: LEVEL

 name	1990Y1	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1	1996Y1	1997Y1	1998Y1	1999Y1	2000Y1
ACAF 	 23.77	 26.24	 30.16	 34.66	  8.16	-13.13	 32.17	 39.94	 29.65	 13.53	 10.05
ACAG 	-28.17	-30.93	-40.29	-43.16	-16.03	-41.85	-40.24	-32.93	-38.35	-39.86	-41.53
AOUC 	  1.00	  1.02	  1.03	  1.03	  1.05	  1.05	  1.05	  1.08	  1.09	  1.11	  1.12
AOUC_	  0.94	  0.96	  0.97	  0.98	  0.99	  1.00	  1.00	  1.03	  1.04	  1.08	  1.10
AQC  	  1.00	  1.06	  1.11	  1.15	  1.16	  1.16	  1.16	  1.20	  1.20	  1.34	  1.34

In [101]:
# get the variable values for a list of non-contiguous periods -> pandas DataFrame
variables["A*", ["1990Y1", "1995Y1", "2000Y1"]]

time,1990Y1,1995Y1,2000Y1
variables,,,
ACAF,23.771000,-13.130997,10.046611
ACAG,-28.172186,-41.845993,-41.534787
AOUC,1.000000,1.049891,1.116238
AOUC_,0.937359,0.995263,1.101957
AQC,1.000000,1.161687,1.338603


In [102]:
# 2) add a variable to the subset
vars_subset["A0"] = np.nan
vars_subset["A0"]              

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A0  	    na	    na	    na	    na	    na	    na	...	    na	    na	    na	    na	    na	    na	    na

In [103]:
# --> new variable also appears in the global workspace
"A0" in variables            

True

In [104]:
variables["A0"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A0  	    na	    na	    na	    na	    na	    na	...	    na	    na	    na	    na	    na	    na	    na

In [105]:
# 3) update a variable in the subset
vars_subset["A0"] = 0.0
vars_subset["A0"]              

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A0  	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00	...	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00

In [106]:
# --> variable is also updated in the global workspace
variables["A0"]                     

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
A0  	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00	...	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00

In [107]:
# delete one variable from a subset of the global database
del vars_subset["A0"]
"A0" in vars_subset

False

In [108]:
# NOTE: the variable has also been deleted from the global database
"A0" in variables

False

It is also possible to work on subset over names and periods:

In [109]:
# 1) get subset over names and periods -> Variables
vars_subset = variables["A*", "1991Y1:1995Y1"]
vars_subset.names

['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [110]:
vars_subset

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

 name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF 	 26.24	 30.16	 34.66	  8.16	-13.13
ACAG 	-30.93	-40.29	-43.16	-16.03	-41.85
AOUC 	  1.02	  1.03	  1.03	  1.05	  1.05
AOUC_	  0.96	  0.97	  0.98	  0.99	  1.00
AQC  	  1.06	  1.11	  1.15	  1.16	  1.16

In [111]:
# 2) update a variable in the subset
vars_subset["ACAF"] = 1.0
vars_subset["ACAF"]

Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1991Y1:1995Y1
mode: LEVEL

name	1991Y1	1992Y1	1993Y1	1994Y1	1995Y1
ACAF	  1.00	  1.00	  1.00	  1.00	  1.00

In [112]:
# --> variable is also updated in the global workspace
variables["ACAF", "1991Y1"]

1.0

In [113]:
variables["ACAF", "1995Y1"]

1.0

<div class="alert alert-info">

**Note**

Selecting non-contiguous periods will return a pandas object, not a Variables object. This is because the IODE Variables workspace is designed to work with contiguous periods. The pandas object is not a view of the Variables workspace, but a copy of the data. Modifying a value of the pandas object will not affect the Variables workspace.

</div>


In [114]:
df_vars_subset = variables["A*", ["1990Y1", "1995Y1", "2000Y1"]]
df_vars_subset

time,1990Y1,1995Y1,2000Y1
variables,,,
ACAF,23.771000,1.000000,10.046611
ACAG,-28.172186,-41.845993,-41.534787
AOUC,1.000000,1.049891,1.116238
AOUC_,0.937359,0.995263,1.101957
AQC,1.000000,1.161687,1.338603


## Estimation

To estimate either one equation or a block of equations, use the [Equation.estimate](../_generated/iode.Equation.estimate.rst#iode.Equation.estimate) method of the equation object.

In [115]:
help(equations.estimate)

Help on method estimate in module iode.iode_cython:

estimate(from_period: 'Union[str, Period]' = None, to_period: 'Union[str, Period]' = None, list_eqs: 'Union[str, List[str]]' = None) method of iode.iode_cython.Equations instance
    estimate(self, from_period: Union[str, Period] = None, to_period: Union[str, Period] = None, list_eqs: Union[str, List[str]] = None)
    
    Estimate an equation or a block of equations.
    
    At the end of the estimation process, certain variables and scalars are automatically created 
    if the process has converged. These variables and scalars can be used for computational purposes and, 
    as they are part of the global workspace, can be saved for future use.
    
    The tests resulting from the last estimation are saved as scalars. The same applies to residuals, 
    left-hand and right-hand members of equations.
    
    Saved tests (as scalars) have the following names (`e<ith>_*` for the <ith> equation of the block):
    
        - `e<ith>

Let's start by reloading all workspaces:

In [116]:
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.cmt
317 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.eqs
274 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.idt
48 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.lst
17 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.scl
161 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.tbl
46 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.var
394 objects loaded


Example for one equation:

In [117]:
# ---- estimate coefficients of one equation ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")

# copy the original values of the coefficients into an isolated scalars workspace
# Useful to restore the original values if the estimation process didn't go well
scalars_copy = scalars[equations['ACAF'].coefficients].copy()
print(f"scalars names in scalars_copy: {scalars_copy.names}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.

# estimate the 'ACAF' equation for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", "ACAF")
# or equivalently
equations["ACAF"].estimate("1980Y1", "1996Y1")

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")

ACAF equation LEC: (ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+
acaf4*(TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
scalars names in scalars_copy: ['acaf1', 'acaf2', 'acaf4']
Estimating : iteration 1 (||eps|| = 0.173205)

Estimating : iteration 2 (||eps|| = 5.16075e-09)

Solution reached after 2 iteration(s). Creating results file ...

Estimating : iteration 1 (||eps|| = 7.05003e-13)

Solution reached after 1 iteration(s). Creating results file ...

Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00136949)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.48247e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.00208257)


Example for a block of equations:

In [118]:
# ---- estimate a block of equations ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")
print(f"DPUH equation LEC: {equations['DPUH'].lec}")
print(f"DPUH equations coefficients (= scalars): {equations['DPUH'].coefficients}")
print(f"DPUH equations variables: {equations['DPUH'].variables}")

# copy the original values of the coefficients into an isolated scalars workspace
# Useful to restore the original values if the estimation process didn't go well
scalars_names = equations['ACAF'].coefficients + equations['DPUH'].coefficients
scalars_names = list(set(scalars_names))   # removed duplicates
scalars_copy = scalars[scalars_names].copy()
print(f"scalars names in scalars_copy: {scalars_copy.names}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.
for name in equations['DPUH'].coefficients:
    scalars[name] = 0., 1.

# prepare equations (same block and method)
block = "ACAF;DPUH"
for name in block.split(";"):
    equations[name] = {"block": block, "method": "LSQ"}

# estimation the block 'ACAF;DPUH' for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", block)

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")
print(f"Resulting values for the coefficient 'dpuh_1': {scalars['dpuh_1']}")
print(f"Resulting values for the coefficient 'dpuh_2': {scalars['dpuh_2']}")

ACAF equation LEC: (ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+
acaf4*(TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
DPUH equation LEC: dln (DPUH/DPUHO):=dpuh_1+dpuh_2*dln(IHU/PI5)+dln PC
DPUH equations coefficients (= scalars): ['dpuh_1', 'dpuh_2']
DPUH equations variables: ['DPUH', 'DPUHO', 'IHU', 'PI5', 'PC']
scalars names in scalars_copy: ['acaf1', 'acaf2', 'acaf4', 'dpuh_1', 'dpuh_2']
Estimating : iteration 1 (||eps|| = 0.223607)

Estimating : iteration 2 (||eps|| = 5.16075e-09)

Solution reached after 2 iteration(s). Creating results file ...

Estimating : iteration 1 (||eps|| = 4.38565e-12)

Solution reached after 1 iteration(s). Creating results file ...

Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00136079)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.47188e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.00206603)


## Simulation

To simulate a model, you must create and initialize an instance of the [Simulation](../_generated/iode.Simulation.rst#iode.Simulation) class:

In [119]:
from iode import Simulation

help(Simulation)

Help on class Simulation in module iode.iode_cython:

class Simulation(builtins.object)
 |  Class for simulate models.
 |  
 |  Models
 |  ~~~~~~
 |  
 |  IN IODE, a model is simply a list of equations. No other construction is necessary: in this way, 
 |  to modify a model, the only thing to do is to modify the list that defines it. 
 |  For example, if a model is logically divided into 5 blocks, 5 lists of equations will be defined:: 
 |  
 |      BLOC1 : A, B, C, D
 |      BLOC2 : X, Y
 |      BLOC3 : C1, C2, C3, C4
 |      BLOC4 : X_A1, X_A2, X_A3
 |      BLOC5 : R1, R2, R3, S_12
 |      MODSIM : $BLOC1, $BLOC2, $BLOC3, $BLOC4, $BLOC5
 |  
 |  To simulate a model, all equations of the model must have been loaded or created. In addition, 
 |  all the variables and scalars used in the model equations must have been loaded or created. 
 |  Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
 |  simulation periods.
 |  
 |  Exchange Endo-Exo
 | 

In [120]:
from iode import SimulationSort

print(f"possible sort algorithms: {[member.name for member in SimulationSort]}")

possible sort algorithms: ['CONNEX', 'BOTH', 'NONE']


In [121]:
from iode import SimulationInitialization

print(f"possible initialization methods: {[member.name for member in SimulationInitialization]}")

possible initialization methods: ['TM1', 'TM1_A', 'EXTRA', 'EXTRA_A', 'ASIS', 'TM1_NA', 'EXTRA_NA']


In [122]:
# ---- simulation ----
print("Simulation() methods and properties:")
for name in dir(Simulation):
    if not name.startswith('_'):
        print(name)

Simulation() methods and properties:
convergence_threshold
debug
debug_newton
initialization_method
max_nb_iterations
max_nb_iterations_newton
model_calculate_SCC
model_compile
model_exchange
model_simulate
model_simulate_SCC
nb_passes
relax
sort_algorithm


In [123]:
# create and initialize a Simulation instance
simu = Simulation(sort_algorithm=SimulationSort.BOTH, initialization_method=SimulationInitialization.TM1)
print(f"Simulation convergence_threshold: {simu.convergence_threshold}")
print(f"Simulation relax: {simu.relax}")
print(f"Simulation max_nb_iterations: {simu.max_nb_iterations}")
print(f"Simulation sort_algorithm: {simu.sort_algorithm}")
print(f"Simulation initialization_method: {simu.initialization_method}")
print(f"Simulation debug: {simu.debug}")
print(f"Simulation nb_passes: {simu.nb_passes}")
print(f"Simulation debug_newton: {simu.debug_newton}")

Simulation convergence_threshold: 0.001
Simulation relax: 1.0
Simulation max_nb_iterations: 100
Simulation sort_algorithm: BOTH (Connex compon. + Triangulation)
Simulation initialization_method: TM1 (Y := Y[-1], if Y null or NA)
Simulation debug: False
Simulation nb_passes: 5
Simulation debug_newton: False


To run the simulation, call the [Simulation.model_simulate](../_generated/iode.Simulation.model_simulate.rst#iode.Simulation.model_simulate) method:

In [124]:
help(Simulation.model_simulate)

Help on cython_function_or_method in module iode.iode_cython:

model_simulate(self, from_period: 'Union[str, Period]', to_period: 'Union[str, Period]', list_eqs: 'Union[str, List[str]]' = None)
    model_simulate(self, from_period: Union[str, Period], to_period: Union[str, Period], list_eqs: Union[str, List[str]] = None)
    
    Run the simulation of a model for a given sample.
    
    To simulate a model, all equations of the model must have been loaded or created. In addition, 
    all the variables and scalars used in the model equations must have been loaded or created. 
    Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
    simulation sample.
    
    Parameters
    ----------
    from_period : str or Period
        The starting period for the simulation.
    to_period : str or Period
        The ending period for the simulation.
    list_eqs : str or list(str), optional
        List of equations representing the model.
        Defau

In [125]:
print(f"exogenous variable 'UY': {equations['UY'].lec}")
print(f"endogenous variable 'XNATY': {identities['XNATY']}")
# reset values of exogenous variable
variables["UY", "2000Y1:2015Y1"] = 0.0

print(f"exogenous variable 'UY' before simulation:\n{variables['UY', '1998Y1:2005Y1']}")

# run the simulation for the periods range '2000Y1' to '2015Y1'
simu.model_simulate("2000Y1", "2015Y1")

print(f"exogenous variable 'UY' after simulation:\n{variables['UY', '1998Y1:2005Y1']}")

exogenous variable 'UY': UY := NATY-NDOMY-NIY-NGY-(EFXY-EFMY)-NFY
endogenous variable 'XNATY': grt NATY
exogenous variable 'UY' before simulation:
Workspace: Variables
nb variables: 1
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1998Y1:2005Y1
mode: LEVEL

name	1998Y1	1999Y1	2000Y1	2001Y1	2002Y1	2003Y1	2004Y1	2005Y1
UY  	671.46	647.39	  0.00	  0.00	  0.00	  0.00	  0.00	  0.00

Linking equations ....
Calculating SCC...
Calculating SCC... 0 ms -> #PRE 31 - #INTER 204 - #POST 39
Reordering interdependent block...
Reordering interdependent block... 0 ms
2000Y1: 1 iters - error =    1.342 - cpu=0ms
2000Y1: 2 iters - error =   0.4115 - cpu=0ms
2000Y1: 3 iters - error =   0.5272 - cpu=0ms
2000Y1: 4 iters - error =   0.0215 - cpu=0ms
2000Y1: 5 iters - error =  0.03879 - cpu=0ms
2000Y1: 6 iters - error =  0.02778 - cpu=0ms
2000Y1: 7 iters - error =   0.1455 - cpu=0ms
2000Y1: 8 iters - error =  0.025

## Import/Export IODE Variables workspace from/to numpy ndarray

To export the content of the `Variables` workspace (or a subset of it) to a numpy ndarray object, use the [to_ndarray](../_generated/iode.Variables.to_ndarray.rst#iode.Variables.to_ndarray) property.

In [126]:
len(variables)

400

In [127]:
variables.sample

Sample("1960Y1:2015Y1")

In [128]:
variables.nb_periods

56

Export the whole Variables workspace to a numpy ndarray:

In [129]:
# export the whole Variables workspace to a numpy ndarray (394 variables x 56 periods)
data = variables.to_ndarray
data.shape

(400, 56)

In [130]:
data[5, 40]

443.91099765099347

In [131]:
variables.i[5, 40]

443.91099765099347

Export a subset of names:

In [132]:
# export a subset of names
vars_subset = variables["A*"]
vars_subset.names


['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [133]:
vars_subset

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 1960Y1:2015Y1
mode: LEVEL

 name	1960Y1	1961Y1	1962Y1	1963Y1	1964Y1	1965Y1	...	2009Y1	2010Y1	2011Y1	2012Y1	2013Y1	2014Y1	2015Y1
ACAF 	    na	    na	    na	    na	    na	    na	...	-37.25	-37.44	-44.19	-55.30	-68.83	-83.44	-96.60
ACAG 	    na	    na	    na	    na	    na	    na	...	 30.01	 31.04	 32.07	 33.10	 34.15	 35.20	 36.25
AOUC 	    na	  0.25	  0.25	  0.26	  0.28	  0.29	...	  1.29	  1.31	  1.33	  1.36	  1.39	  1.42	  1.46
AOUC_	    na	    na	    na	    na	    na	    na	...	  1.23	  1.25	  1.27	  1.30	  1.34	  1.37	  1.41
AQC  	  0.22	  0.22	  0.22	  0.23	  0.24	  0.25	...	  1.45	  1.46	  1.48	  1.51	  1.56	  1.61	  1.67

In [134]:
data = vars_subset.to_ndarray
data.shape

(5, 56)

In [135]:
# values of the 'ACAF' variable
data[0]

array([         nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,   1.2130001 ,   5.2020001 ,
         9.184     ,   8.0790005 ,  11.332     ,  13.518001  ,
        15.784     ,  16.544001  ,  21.489     ,  20.281     ,
        21.277     ,  32.417999  ,  24.446999  ,  27.025002  ,
        24.504     ,  27.560999  ,  25.542     ,  27.499001  ,
        25.353001  ,  17.165001  ,  23.771     ,  26.240999  ,
        30.159     ,  34.661999  ,   8.1610022 , -13.130997  ,
        32.171001  ,  39.935799  ,  29.645657  ,  13.53040492,
        10.02457312,   2.59495261,  -1.30484331,  -6.43120286,
       -14.89734264, -26.86600502, -29.23931857, -33.52355331,
       -38.53077501, -37.25323   , -37.44456794, -44.19241697,
       -55.30055074, -68.83211976, -83.43527765, -96.59539214])

In [136]:
# values of the 'AQC' variable
data[-1]

array([ 0.21753037,  0.21544869,  0.22228125,  0.22953896,  0.23653506,
        0.24732406,  0.26255098,  0.26907021,  0.27206925,  0.27986595,
        0.29396999,  0.31906503,  0.3426649 ,  0.36655167,  0.42489415,
        0.49478459,  0.53812659,  0.5841772 ,  0.61441606,  0.64528418,
        0.68947881,  0.73596764,  0.77532566,  0.82384807,  0.85829282,
        0.90006256,  0.92794591,  0.93221092,  0.92874223,  0.9445076 ,
        1.        ,  1.0628064 ,  1.1102825 ,  1.1532652 ,  1.1571276 ,
        1.1616869 ,  1.1580297 ,  1.201328  ,  1.2031082 ,  1.34296997,
        1.33860286,  1.37918825,  1.40881647,  1.41970458,  1.40065206,
        1.39697298,  1.39806354,  1.40791334,  1.42564488,  1.44633167,
        1.46286837,  1.48227361,  1.51366598,  1.55803879,  1.61318117,
        1.67429058])

Export a subset of names and periods:

In [137]:
# export a subset of names and periods
vars_subset = variables["A*", "2000Y1:2010Y1"]
vars_subset

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	2000Y1	2001Y1	2002Y1	2003Y1	2004Y1	2005Y1	2006Y1	2007Y1	2008Y1	2009Y1	2010Y1
ACAF 	 10.02	  2.59	 -1.30	 -6.43	-14.90	-26.87	-29.24	-33.52	-38.53	-37.25	-37.44
ACAG 	-38.82	 21.72	 22.76	 23.80	 24.85	 25.89	 26.91	 27.94	 28.97	 30.01	 31.04
AOUC 	  1.12	  1.14	  1.16	  1.17	  1.17	  1.18	  1.20	  1.22	  1.26	  1.29	  1.31
AOUC_	  1.10	  1.14	  1.15	  1.16	  1.15	  1.16	  1.19	  1.20	  1.21	  1.23	  1.25
AQC  	  1.34	  1.38	  1.41	  1.42	  1.40	  1.40	  1.40	  1.41	  1.43	  1.45	  1.46

In [138]:
data = vars_subset.to_ndarray
data.shape

(5, 11)

In [139]:
data

array([[ 10.02457312,   2.59495261,  -1.30484331,  -6.43120286,
        -14.89734264, -26.86600502, -29.23931857, -33.52355331,
        -38.53077501, -37.25323   , -37.44456794],
       [-38.81610968,  21.72129465,  22.76221167,  23.80234137,
         24.84866767,  25.89044634,  26.91215138,  27.94337852,
         28.97461821,  30.01172765,  31.03508786],
       [  1.11561357,   1.14021801,   1.15748416,   1.17126518,
          1.16873575,   1.18281782,   1.20084854,   1.22073362,
          1.2642839 ,   1.2883006 ,   1.30796672],
       [  1.1019572 ,   1.13624426,   1.15021519,   1.16082895,
          1.14802147,   1.16412337,   1.18589708,   1.19516611,
          1.21383423,   1.23185399,   1.25016433],
       [  1.33860286,   1.37918825,   1.40881647,   1.41970458,
          1.40065206,   1.39697298,   1.39806354,   1.40791334,
          1.42564488,   1.44633167,   1.46286837]])

To update a subset of the Variables workspace, use the [from_ndarray](../_generated/iode.Variables.from_ndarray.rst#iode.Variables.from_ndarray) method.

In [140]:
vars_names = variables.get_names("A*")
vars_names

['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [141]:
first_period = "2000Y1"
last_periods = "2010Y1"
sample = Sample(first_period, last_periods)
nb_periods = sample.nb_periods
nb_periods

11

In [142]:
# save original values to restore them later
original_values = variables["A*", "2000Y1:2010Y1"].to_ndarray
original_values

array([[ 10.02457312,   2.59495261,  -1.30484331,  -6.43120286,
        -14.89734264, -26.86600502, -29.23931857, -33.52355331,
        -38.53077501, -37.25323   , -37.44456794],
       [-38.81610968,  21.72129465,  22.76221167,  23.80234137,
         24.84866767,  25.89044634,  26.91215138,  27.94337852,
         28.97461821,  30.01172765,  31.03508786],
       [  1.11561357,   1.14021801,   1.15748416,   1.17126518,
          1.16873575,   1.18281782,   1.20084854,   1.22073362,
          1.2642839 ,   1.2883006 ,   1.30796672],
       [  1.1019572 ,   1.13624426,   1.15021519,   1.16082895,
          1.14802147,   1.16412337,   1.18589708,   1.19516611,
          1.21383423,   1.23185399,   1.25016433],
       [  1.33860286,   1.37918825,   1.40881647,   1.41970458,
          1.40065206,   1.39697298,   1.39806354,   1.40791334,
          1.42564488,   1.44633167,   1.46286837]])

In [143]:
# create the numpy ndarray containing the values to copy into the Variables database
data = np.zeros((len(vars_names), nb_periods), dtype=float)
for i in range(len(vars_names)):
    for j in range(nb_periods):
        data[i, j] = i * nb_periods + j
data

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.],
       [ 11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.],
       [ 22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.],
       [ 33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.],
       [ 44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.]])

In [144]:
variables["A*", "2000Y1:2010Y1"]

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	2000Y1	2001Y1	2002Y1	2003Y1	2004Y1	2005Y1	2006Y1	2007Y1	2008Y1	2009Y1	2010Y1
ACAF 	 10.02	  2.59	 -1.30	 -6.43	-14.90	-26.87	-29.24	-33.52	-38.53	-37.25	-37.44
ACAG 	-38.82	 21.72	 22.76	 23.80	 24.85	 25.89	 26.91	 27.94	 28.97	 30.01	 31.04
AOUC 	  1.12	  1.14	  1.16	  1.17	  1.17	  1.18	  1.20	  1.22	  1.26	  1.29	  1.31
AOUC_	  1.10	  1.14	  1.15	  1.16	  1.15	  1.16	  1.19	  1.20	  1.21	  1.23	  1.25
AQC  	  1.34	  1.38	  1.41	  1.42	  1.40	  1.40	  1.40	  1.41	  1.43	  1.45	  1.46

In [145]:
# copy the numpy ndarray into the Variables database (overriding the existing values)
variables.from_ndarray(data, vars_names, first_period, last_periods)
variables["A*", "2000Y1:2010Y1"]

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	2000Y1	2001Y1	2002Y1	2003Y1	2004Y1	2005Y1	2006Y1	2007Y1	2008Y1	2009Y1	2010Y1
ACAF 	  0.00	  1.00	  2.00	  3.00	  4.00	  5.00	  6.00	  7.00	  8.00	  9.00	 10.00
ACAG 	 11.00	 12.00	 13.00	 14.00	 15.00	 16.00	 17.00	 18.00	 19.00	 20.00	 21.00
AOUC 	 22.00	 23.00	 24.00	 25.00	 26.00	 27.00	 28.00	 29.00	 30.00	 31.00	 32.00
AOUC_	 33.00	 34.00	 35.00	 36.00	 37.00	 38.00	 39.00	 40.00	 41.00	 42.00	 43.00
AQC  	 44.00	 45.00	 46.00	 47.00	 48.00	 49.00	 50.00	 51.00	 52.00	 53.00	 54.00

If you already work on the subset you whish to update the values, you can skip to specify the value for the parameters *vars_names*, *first_period* and *last_period*:

In [146]:
vars_subset = variables["A*", "2000Y1:2010Y1"]
vars_subset.from_ndarray(original_values)
vars_subset

Workspace: Variables
nb variables: 5
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	2000Y1	2001Y1	2002Y1	2003Y1	2004Y1	2005Y1	2006Y1	2007Y1	2008Y1	2009Y1	2010Y1
ACAF 	 10.02	  2.59	 -1.30	 -6.43	-14.90	-26.87	-29.24	-33.52	-38.53	-37.25	-37.44
ACAG 	-38.82	 21.72	 22.76	 23.80	 24.85	 25.89	 26.91	 27.94	 28.97	 30.01	 31.04
AOUC 	  1.12	  1.14	  1.16	  1.17	  1.17	  1.18	  1.20	  1.22	  1.26	  1.29	  1.31
AOUC_	  1.10	  1.14	  1.15	  1.16	  1.15	  1.16	  1.19	  1.20	  1.21	  1.23	  1.25
AQC  	  1.34	  1.38	  1.41	  1.42	  1.40	  1.40	  1.40	  1.41	  1.43	  1.45	  1.46

## Import/Export IODE workspaces from/to pandas Series and DataFrame

To import / export the content of the `comments`, `identities` and `lists` workspaces from/to a pandas Series object, use the [from_series()](../_generated/iode.Comments.from_series.rst#iode.Comments.from_series) and [to_series()](../_generated/iode.Comments.to_series.rst#iode.Comments.to_series) methods. 

Alternatively, you can use the [series](../_generated/iode.Comments.series.rst#iode.Comments.series) property to export the content of the `comments`, `identities` and `lists` workspaces to a pandas Series object.

In [147]:
import pandas as pd

# ---- to pandas Series ----
# See Comments/Identities/Lists.to_series and 
#     Comments/Identities/Lists.series

series_cmt = comments.to_series()
print(f"Comments as pandas Series:\n{series_cmt.info()}")
print()

series_idt = identities.to_series()
print(f"Identities as pandas Series:\n{series_idt.info()}")
print()

series_lst = lists.to_series()
print(f"Lists as pandas Series:\n{series_lst.info()}")

# Alternatively

series_cmt = comments.series
series_idt = identities.series
series_lst = lists.series

# ---- from pandas Series ----
# See Comments/Identities/Lists.from_series

comments.from_series(series_cmt)
identities.from_series(series_idt)
lists.from_series(series_lst)

Comments as pandas Series:
<bound method Series.info of ACAF        Ondernemingen: ontvangen kapitaaloverdrachten.
ACAG     Totale overheid: netto ontvangen kapitaaloverd...
AOUC                             Kost per eenheid produkt.
AQC          Kost per eenheid produkt: kapitaal en arbeid.
BENEF             Ondernemingen: niet-uitgekeerde winsten.
                               ...                        
ZF       Indexeringscoëfficiënt voor de lonen in de pri...
ZJ       Indexeringscoëfficiënt voor de sociale uitkeri...
ZKF            Bezettingsgraad van de produktiecapaciteit.
ZX                          Saut d'index (correction en %)
ZZ_      Marktsector (ondernemingen en zelfstandigen): ...
Name: Comments, Length: 317, dtype: object>

Identities as pandas Series:
<bound method Series.info of AOUC      ((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+VAFF))...
AOUC_     exp(ln(((WCF/NFYH)/QL)+PKF/(QAFF/KNFFY))*(QAFF...
FLGR                                               FLG/VBBP
GAP2        

To import / export the content of the `equations`, `scalars` and `variables` workspaces from/to a pandas DataFrame object, use the [from_frame()](../_generated/iode.Variables.from_frame.rst#iode.Variables.from_frame) and [to_frame()](../_generated/iode.Variables.to_frame.rst#iode.Variables.to_frame) methods. 

Alternatively, you can use the [df](../_generated/iode.Variables.df.rst#iode.Variables.df) property to export the content of the `equations`, `scalars` and `variables` workspaces to a pandas DataFrame object.

In [148]:
# ---- to pandas DataFrame ----
# See Equations/Scalars/Variables.to_frame and
#     Equations/Scalars/Variables.df

df_eqs = equations.to_frame()
print(f"Equations as pandas DataFrame:\n{df_eqs.info()}")
print()

df_scl = scalars.to_frame()
print(f"Scalars as pandas DataFrame:\n{df_scl.info()}")
print()

df_vars = variables.to_frame()
print(f"Variables as pandas DataFrame:\n{df_vars.info()}")

# Alternatively

df_eqs = equations.df
df_scl = scalars.df
df_vars = variables.df

# ---- from pandas DataFrame ----
# See Equations/Scalars/Variables.from_frame

equations.from_frame(df_eqs)
scalars.from_frame(df_scl)
variables.from_frame(df_vars)

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, ACAF to ZZF_
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lec          274 non-null    object 
 1   method       274 non-null    object 
 2   sample       274 non-null    object 
 3   comment      274 non-null    object 
 4   instruments  274 non-null    object 
 5   block        274 non-null    object 
 6   corr         274 non-null    float64
 7   stdev        274 non-null    float64
 8   meany        274 non-null    float64
 9   ssres        274 non-null    float64
 10  stderr       274 non-null    float64
 11  stderrp      274 non-null    float64
 12  fstat        274 non-null    float64
 13  r2           274 non-null    float64
 14  r2adj        274 non-null    float64
 15  dw           274 non-null    float64
 16  loglik       274 non-null    float64
 17  date         274 non-null    object 
dtypes: float64(11), object(7)
memory usage: 40.7+ KB
Eq

C:\Users\ALD\AppData\Local\Temp\ipykernel_35336\3370860065.py:5: UserWarning: 'sample' is not defined
  df_eqs = equations.to_frame()
C:\Users\ALD\AppData\Local\Temp\ipykernel_35336\3370860065.py:18: UserWarning: 'sample' is not defined
  df_eqs = equations.df


It possible to export a subset of the IODE databases:

In [149]:
# export a subset of the IODE Variables database as a pandas DataFrame
vars_subset = variables["A*;*_", "2000Y1:2010Y1"]
vars_subset

Workspace: Variables
nb variables: 33
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	 2000Y1	 2001Y1	 2002Y1	 2003Y1	 2004Y1	 2005Y1	 2006Y1	 2007Y1	 2008Y1 	 2009Y1 	 2010Y1 
ACAF 	  10.02	   2.59	  -1.30	  -6.43	 -14.90	 -26.87	 -29.24	 -33.52	  -38.53	  -37.25	  -37.44
ACAG 	 -38.82	  21.72	  22.76	  23.80	  24.85	  25.89	  26.91	  27.94	   28.97	   30.01	   31.04
AOUC 	   1.12	   1.14	   1.16	   1.17	   1.17	   1.18	   1.20	   1.22	    1.26	    1.29	    1.31
AOUC_	   1.10	   1.14	   1.15	   1.16	   1.15	   1.16	   1.19	   1.20	    1.21	    1.23	    1.25
AQC  	   1.34	   1.38	   1.41	   1.42	   1.40	   1.40	   1.40	   1.41	    1.43	    1.45	    1.46
...  	    ...	    ...	    ...	    ...	    ...	    ...	    ...	    ...	     ...	     ...	     ...
WCF_ 	3713.86	3865.19	4005.24	4157.26	4254.81	4334.99	4478.85	4682.26	 4931.85	 5054.42	 5179.35
WIND_	 999.00	10

In [150]:
df = vars_subset.to_frame()
df.shape

(33, 11)

In [151]:
df

time,2000Y1,2001Y1,2002Y1,2003Y1,2004Y1,2005Y1,2006Y1,2007Y1,2008Y1,2009Y1,2010Y1
names,,,,,,,,,,,
ACAF,10.024573,2.594953,-1.304843,-6.431203,-14.897343,-26.866005,-29.239319,-33.523553,-38.530775,-37.253230,-37.444568
ACAG,-38.816110,21.721295,22.762212,23.802341,24.848668,25.890446,26.912151,27.943379,28.974618,30.011728,31.035088
AOUC,1.115614,1.140218,1.157484,1.171265,1.168736,1.182818,1.200849,1.220734,1.264284,1.288301,1.307967
AOUC_,1.101957,1.136244,1.150215,1.160829,1.148021,1.164123,1.185897,1.195166,1.213834,1.231854,1.250164
AQC,1.338603,1.379188,1.408816,1.419705,1.400652,1.396973,1.398064,1.407913,1.425645,1.446332,1.462868
GAP_,4.510096,3.321449,2.647524,3.530320,5.569508,5.716970,6.022618,6.017820,4.944350,3.823078,3.376524
GOSH_,1957.089520,2067.979243,2152.594802,2238.977778,2342.701343,2440.576276,2493.549081,2571.415491,2672.317047,2771.485245,2836.227715
PAF_,1.197151,1.229645,1.260528,1.287127,1.297059,1.287859,1.284935,1.289950,1.306855,1.327965,1.344200
PC_,1.234475,1.268933,1.294223,1.313455,1.327835,1.350464,1.372854,1.396742,1.445474,1.472675,1.495632


## Import/Export the Variables workspace from/to LArray Array

To import / export the content of the `variables` workspaces from/to a LArray Array object, use the [from_array()](../_generated/iode.Variables.from_array.rst#iode.Variables.from_array) and [to_array()](../_generated/iode.Variables.to_array.rst#iode.Variables.to_array) methods:

In [152]:
import larray as la

# ---- to LArray Array ----
# See Variables.to_array

arr_vars = variables.to_array()
print(f"Variables as LArray Array:\n{arr_vars.info}")

# ---- from LArray array ----
# See Variables.from_array

variables.from_array(arr_vars)

Variables as LArray Array:
400 x 56
 names [400]: 'ACAF' 'ACAG' 'AOUC' ... '_YOBS1' '_YRES0' '_YRES1'
 time [56]: '1960Y1' '1961Y1' '1962Y1' ... '2013Y1' '2014Y1' '2015Y1'
dtype: float64
memory used: 175.00 Kb


It possible to export a subset of the IODE databases:

In [153]:
# export a subset of the IODE Variables database as a LArray Array
vars_subset = variables["A*;*_", "2000Y1:2010Y1"]
vars_subset

Workspace: Variables
nb variables: 33
filename: c:\soft\Miniconda3\Lib\site-packages\iode\tests\data\fun.var
description: Modèle fun - Simulation 1                         
sample: 2000Y1:2010Y1
mode: LEVEL

 name	 2000Y1	 2001Y1	 2002Y1	 2003Y1	 2004Y1	 2005Y1	 2006Y1	 2007Y1	 2008Y1 	 2009Y1 	 2010Y1 
ACAF 	  10.02	   2.59	  -1.30	  -6.43	 -14.90	 -26.87	 -29.24	 -33.52	  -38.53	  -37.25	  -37.44
ACAG 	 -38.82	  21.72	  22.76	  23.80	  24.85	  25.89	  26.91	  27.94	   28.97	   30.01	   31.04
AOUC 	   1.12	   1.14	   1.16	   1.17	   1.17	   1.18	   1.20	   1.22	    1.26	    1.29	    1.31
AOUC_	   1.10	   1.14	   1.15	   1.16	   1.15	   1.16	   1.19	   1.20	    1.21	    1.23	    1.25
AQC  	   1.34	   1.38	   1.41	   1.42	   1.40	   1.40	   1.40	   1.41	    1.43	    1.45	    1.46
...  	    ...	    ...	    ...	    ...	    ...	    ...	    ...	    ...	     ...	     ...	     ...
WCF_ 	3713.86	3865.19	4005.24	4157.26	4254.81	4334.99	4478.85	4682.26	 4931.85	 5054.42	 5179.35
WIND_	 999.00	10

In [154]:
array = vars_subset.to_array()
array.shape

(33, 11)

In [155]:
array

names\time               2000Y1  ...               2010Y1
      ACAF   10.024573120674221  ...   -37.44456793550647
      ACAG   -38.81610967500789  ...    31.03508786125745
      AOUC   1.1156135743606619  ...    1.307966721108404
     AOUC_     1.10195719812178  ...   1.2501643331956398
       AQC   1.3386028553645442  ...   1.4628683697450802
      GAP_    4.510095736743436  ...    3.376524129115177
     GOSH_   1957.0895201725107  ...    2836.227715391554
      PAF_   1.1971514192251522  ...   1.3442002853082462
       PC_     1.23447486130748  ...    1.495632173503789
      PFI_   1.0733284895365958  ...   1.2930032464932164
   PROIHP_            1.1054128  ...           0.95035726
     QAFF_    5751.811754387035  ...     6972.66228157539
      QAF_    4788.946995113995  ...    5934.863147668144
      QAI_    962.9022538979053  ...   1037.6707663907387
      QAT_    7194.830907535022  ...    8725.123886684794
  QBBPPOT_    7662.966196451325  ...    9605.325099920312
       QC_    

## Execute IODE report commands/files

To run an IODE command, use the [execute_command()](../_generated/iode.execute_command.rst#iode.execute_command) function:

In [156]:
from iode import execute_command

# ---- execute IODE commands ----
print("Execute IODE commands (useful for IODE functions and commands not yet ported to Python):")
execute_command("$WsClearVar")
execute_command("$WsSample 2000Y1 2005Y1")
execute_command("$DataCalcVar A t+1")
execute_command("$DataCalcVar B t-1")
execute_command("$DataCalcVar C A/B")
execute_command("$DataCalcVar D grt A")
execute_command("$WsSaveVar test_var.av")
with open("test_var.av", "r") as f:
    print(f.read())

Execute IODE commands (useful for IODE functions and commands not yet ported to Python):
Saving test_var.av
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 



To run an entire IODE report (i.e. file with the '.rep' extension), call the [execute_report()](../_generated/iode.execute_report.rst#iode.execute_report) function:

In [157]:
from iode import execute_report

# ---- execute IODE reports ----
print("Execute an IODE report -> execute_report()")
with open("create_var.rep", "w") as f:
    f.write("$WsClearVar\n")
    f.write("$WsSample 2000Y1 2005Y1\n")
    f.write("$DataCalcVar %1% t+1 \n")
    f.write("$DataCalcVar %2% t-1 \n")
    f.write("$DataCalcVar %3% %1%/%2%\n")
    f.write("$DataCalcVar %4% grt %1% \n")
    f.write("$WsSaveVar test_var.av\n")

execute_report("create_var.rep", ["A", "B", "C", "D"])

with open("test_var.av", "r") as f:
    print(f.read())


Execute an IODE report -> execute_report()
Saving test_var.av
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 

